In [8]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [10]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data
import xarray as xr
import h5py
import pickle

# Helpful
import time
import datetime
import itertools
from itertools import product
from tqdm import tqdm
import os

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp

<IPython.core.display.Javascript object>

### Goal of this notebook: Train EMOS global
1. Train one EMOS model globally on the whole grid for each lead time (1...31) and variable(0...4) total 155 Models

#### 0. Basics

In [11]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

<IPython.core.display.Javascript object>

In [12]:
# Variables to make testing easier --> determines variable and lead_time
var = 0  # 0,...,4
lead_time = 0  # 0,...,30

<IPython.core.display.Javascript object>

#### 1. Load Dataset

In [13]:
dat_train_proc = ldp.load_data_all_train_proc_norm()
dat_test_proc = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

#### 2. Data Split

In [14]:
X_train_var_lead_all, y_train_var_lead_all = split_var_lead(dat_train_proc)
X_test_lead_all, y_test_var_lead_all = split_var_lead(dat_test_proc)

<IPython.core.display.Javascript object>

#### 3. Load Baseline Scores

In [15]:
# Load all scores:
with open(
    "/Data/Delong_BA_Data/scores/crps_benchmark_scores/crps_var_lead_test.pkl", "rb"
) as f:
    crps_var_lead_test_scores = pickle.load(f)

<IPython.core.display.Javascript object>

In [16]:
# Load mean scores:
with open(
    "/Data/Delong_BA_Data/scores/crps_benchmark_scores/crps_var_lead_mean_test.pkl",
    "rb",
) as f:
    crps_var_lead_mean_test_scores = pickle.load(f)

<IPython.core.display.Javascript object>

#### 4. Load Emos Models

In [19]:
def main():
    """
    Load var_lead_time EMOS Models 5x31
    """
    var_names = ["u10", "v10", "t2m", "t850", "z500"]
    EMOS_global_var_lead_models = [[], [], [], [], []]
    for var in range(5):
        for lead_time in range(31):
            EMOS_global_var_lead_models[var].append(
                tf.keras.models.load_model(
                    "/home/dchen/BA_CH_EN/models/EMOS_global_models/EMOS_glob_"
                    + var_names[var]
                    + "_lead_time_"
                    + str(lead_time)
                    + ".h5",
                    custom_objects={"crps_cost_function": crps_cost_function},
                )
            )
    return EMOS_global_var_lead_models

<IPython.core.display.Javascript object>

In [20]:
EMOS_global_var_lead_models = main()

<IPython.core.display.Javascript object>

In [22]:
len(EMOS_global_var_lead_models[0])

31

<IPython.core.display.Javascript object>

In [23]:
len(EMOS_global_var_lead_models)

5

<IPython.core.display.Javascript object>

In [108]:
preds = EMOS_glob.predict(
    [
        X_test_lead_all[var][lead_time].isel(mean_std=0).values.flatten(),
        X_test_lead_all[var][lead_time].isel(mean_std=1).values.flatten(),
    ],
    verbose=1,
)

174038/174038 [==============================] - 226s 1ms/step


<IPython.core.display.Javascript object>

In [111]:
crps_emos = crps_normal(
    mu=preds[:, 0],
    sigma=preds[:, 1],
    y=y_test_var_lead_all[var][lead_time].values.flatten(),
)

<IPython.core.display.Javascript object>

In [112]:
crps_emos.mean()

0.010947947142388477

<IPython.core.display.Javascript object>